In [12]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%autoreload 2    
import os, glob, re
import zipfile as zf
import WIDParseTools as wpt
import pandas as pd
import time

In [26]:
mypath = '<<<ENTER YOUR PATH HERE>>>\\Documents\\My SAP BusinessObjects Documents\\userDocs\\'
myWebiReport = '<<<REPORTNAME>>>.wid'

myWebiPathReport =  mypath + myWebiReport

In [27]:
docspecstring = wpt.getReportXMLString(myWebiPathReport)
xmldoc = wpt.getReportXML(myWebiPathReport)

# print(docspecstring)

In [28]:
# %%time

print( 'analyzing report', myWebiReport)
print()

# Get all report variables
print( 'retrieving report variables')
allvarlist = wpt.getAllReportVariables(myWebiPathReport)
# df_vars = pd.DataFrame(allvarlist, columns =['ID', 'Name', 'Type', 'Universe','Info'])
# print(df_vars.value_counts(subset=['Type'], ascending=False ).to_string())

# print( 'found', len(allvarlist), 'variables')

# print (allvarlist)

analyzing report LFI024 Loonkostenprognose(0).wid

retrieving report variables
closing the file


In [29]:
%%time

# Get all dependencies
print()
print( 'retrieving dependencies...')
vardeps = wpt.getAllVariableDepencencies(myWebiPathReport, allvarlist)
print( 'done')
# Get shortest path from var to reporth
vardepslist = wpt.getReportVarsAndDependencies(allvarlist, vardeps)

df_vardependencies = pd.DataFrame(vardepslist, columns =['Var Nr', 'ID', 'Name', 'Type', 'Info',  'DP ID', 'Universe','Report dependency', 'Path', 'Count'])
# print('created var dependencies')
print(df_vardependencies.value_counts(subset=['Type', 'Report dependency'], sort=False).to_string() )

print()


retrieving dependencies...
done
Type                    Report dependency
DataProvider Object     Ja                   35
                        Nee                  16
Report Variable         Ja                   31
                        Nee                   4
Samengevoegde dimensie  Ja                    5
                        Nee                   2

CPU times: total: 234 ms
Wall time: 254 ms


In [30]:
%%time 
# Get all xpaths of report vars
varreportusage = []

xpaths = wpt.getReportXPaths(xmldoc)

for i, myvar in enumerate(allvarlist):
    varID = myvar[0]

    varpaths = []
    for varpath in xpaths:
        foundvars = varpath[4]
        if varID in foundvars:
            varpaths.append(varpath[0])

    # remove duplicates
    varpaths = list(set(varpaths))
    
    for xpath in varpaths:
        varreportusage.append((myvar[0], myvar[1], myvar[2], myvar[4], xpath))

df_varreportusage = pd.DataFrame(varreportusage, columns =['Var ID', 'Name', 'Type', 'Info', 'Report XPath'])


CPU times: total: 141 ms
Wall time: 129 ms


In [31]:
# Get alerter status
alert_stat = wpt.getAlerterStatus(myWebiPathReport)
df_alerters = pd.DataFrame(alert_stat, columns =['Alerter ID', 'Alerter Name', 'Status'])
print('created alerter usage')

# Write results to excel
with pd.ExcelWriter(mypath + myWebiReport + '.xlsx') as excel_writer:
    df_vardependencies.to_excel(excel_writer, sheet_name='Variable dependencies', index=False)
    df_varreportusage.to_excel(excel_writer, sheet_name='Variable report usage', index=False)
    df_alerters.to_excel(excel_writer, sheet_name='Alerter status', index=False)

print( 'export to excel complete')

created alerter usage
export to excel complete
